<a href="https://colab.research.google.com/github/Satyake/NLP/blob/master/yes_no_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np 

In [2]:
with open('/content/train_qa.txt','rb') as f:
  train_data=pickle.load(f)

In [3]:
with open('/content/train_qa.txt','rb') as f:
  test_data=pickle.load(f)

In [4]:

#train_data=np.array(train_data)

In [5]:
#train_data.shape

In [6]:
set(train_data[0][0])

{'.',
 'Mary',
 'Sandra',
 'bathroom',
 'bedroom',
 'journeyed',
 'moved',
 'the',
 'to'}

In [7]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [8]:
''.join(train_data[0][2])

'no'

In [9]:
all_data=test_data+train_data

In [10]:
len(all_data)

20000

In [11]:
#using set to add words.
vocab=set()
for story ,question, answer in all_data:
  vocab=vocab.union(set(story))
  vocab=vocab.union(set(question))


In [12]:
vocab.add('no')
vocab.add('yes')

In [13]:
vocab_len=len(vocab)+1

In [14]:
all_story_len=[len(data[0]) for data in all_data]
all_question_len=[len(data[1]) for data in all_data] 

In [15]:
max_story_len=max(all_story_len)
max_question_len=max(all_question_len)

In [16]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [17]:
tknizer=Tokenizer(filters=[])

In [18]:
tknizer.fit_on_texts(vocab)

In [19]:
tknizer.word_index

{'.': 18,
 '?': 30,
 'apple': 20,
 'back': 34,
 'bathroom': 2,
 'bedroom': 3,
 'daniel': 5,
 'discarded': 36,
 'down': 33,
 'dropped': 12,
 'football': 10,
 'garden': 35,
 'got': 6,
 'grabbed': 21,
 'hallway': 7,
 'in': 16,
 'is': 22,
 'john': 29,
 'journeyed': 9,
 'kitchen': 32,
 'left': 17,
 'mary': 1,
 'milk': 37,
 'moved': 4,
 'no': 13,
 'office': 31,
 'picked': 11,
 'put': 27,
 'sandra': 8,
 'the': 15,
 'there': 28,
 'to': 26,
 'took': 19,
 'travelled': 25,
 'up': 23,
 'went': 14,
 'yes': 24}

In [20]:
#perform tokenizing for story question and answer
train_story_text=[]
train_question_text=[]
train_answers=[]
for story,question,answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
  train_answers.append(answer)

In [21]:
train_story_seq=tknizer.texts_to_sequences(train_story_text)

In [22]:
pad_sequences(train_story_seq,maxlen=max_story_len)

array([[ 0,  0,  0, ..., 15,  3, 18],
       [ 0,  0,  0, ..., 15,  7, 18],
       [ 0,  0,  0, ..., 15,  2, 18],
       ...,
       [ 0,  0,  0, ..., 15,  3, 18],
       [ 0,  0,  0, ..., 37, 28, 18],
       [ 0,  0,  0, ..., 20, 28, 18]], dtype=int32)

In [23]:
max_story_len

156

In [24]:
#word_index=tknizer.word_index

In [25]:
#function for vectorize
def vectorize_stories(data,word_index=tknizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
  #stories
  X=[]
  #questions
  Xq=[]
  #answers Y/N
  Y=[]
  for story,query,answer in data:
    x=[word_index[word.lower()] for word in story]
    xq=[word_index[word.lower()] for word in query]
    y=np.zeros(len(word_index)+1)
    y[word_index[answer]]=1
    X.append(x)
    Xq.append(xq)
    Y.append(y)
  return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))



In [26]:
max_question_len

6

In [27]:
inputs_train,queries_train,answers_train=vectorize_stories(train_data)
inputs_test,queries_test,answers_test=vectorize_stories(test_data)

In [28]:
inputs_train

array([[ 0,  0,  0, ..., 15,  3, 18],
       [ 0,  0,  0, ..., 15,  7, 18],
       [ 0,  0,  0, ..., 15,  2, 18],
       ...,
       [ 0,  0,  0, ..., 15,  3, 18],
       [ 0,  0,  0, ..., 37, 28, 18],
       [ 0,  0,  0, ..., 20, 28, 18]], dtype=int32)

In [29]:
tknizer.word_index['no']

13

In [30]:
from keras.models import Sequential,Model
from keras.layers.embeddings import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout,LSTM,concatenate,dot,add

In [31]:
#Placeholder shape=(max_story_len,batch_size)
input_sequence=Input((max_story_len,))
question=Input((max_question_len,))

In [32]:
#vocab_len 
vocab_size=len(vocab)+1

In [33]:
#Input Encoder M
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.5))
#outputs (samples,story_maxlen,embedding_dim)


In [34]:
#Input Encoder C
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.5))
#outputs (samples,story_maxlen,embedding_dim)


In [35]:
#Input Encoder Q
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.5))
#outputs (samples,story_maxlen,max_question_len)



In [36]:
input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [37]:
match=dot([input_encoded_m,question_encoded],axes=(2,2))
match=Activation('softmax')(match)

In [38]:
response=add([match,input_encoded_c])
response=Permute((2,1))(response)

In [39]:
answer=concatenate([response,question_encoded])

In [40]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [41]:
answer=LSTM(100)(answer)

In [42]:
answer

<KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm')>

In [43]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_size)(answer)

In [44]:
answer=Activation('softmax')(answer)


In [45]:
model=Model([input_sequence,question],answer)

In [46]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [47]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 156)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, 64)     2432        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 6, 64)        2432        input_2[0][0]                    
______________________________________________________________________________________________

In [48]:
history=model.fit([inputs_train,queries_train],answers_train,batch_size=64,epochs=100,validation_data=([inputs_test,queries_test],answers_test))

Epoch 1/100
157/157 [==============================] - 9s 43ms/step - loss: 1.3465 - accuracy: 0.4637 - val_loss: 0.7008 - val_accuracy: 0.4988
Epoch 2/100
157/157 [==============================] - 6s 37ms/step - loss: 0.7186 - accuracy: 0.4896 - val_loss: 0.6943 - val_accuracy: 0.5012
Epoch 3/100
157/157 [==============================] - 6s 37ms/step - loss: 0.7023 - accuracy: 0.4964 - val_loss: 0.6937 - val_accuracy: 0.4988
Epoch 4/100
157/157 [==============================] - 6s 37ms/step - loss: 0.7002 - accuracy: 0.4881 - val_loss: 0.6963 - val_accuracy: 0.5012
Epoch 5/100
157/157 [==============================] - 6s 37ms/step - loss: 0.6967 - accuracy: 0.5008 - val_loss: 0.6936 - val_accuracy: 0.5012
Epoch 6/100
157/157 [==============================] - 6s 38ms/step - loss: 0.6957 - accuracy: 0.5077 - val_loss: 0.6945 - val_accuracy: 0.5012
Epoch 7/100
157/157 [==============================] - 6s 38ms/step - loss: 0.6975 - accuracy: 0.4919 - val_loss: 0.6996 - val_accuracy:

In [50]:
model.save('Epoch100.h5')

In [51]:
predicted=model.predict([inputs_test,queries_test])

In [52]:
predicted.shape 

(10000, 38)

In [69]:
test_data[0][0]

['Mary',
 'moved',
 'to',
 'the',
 'bathroom',
 '.',
 'Sandra',
 'journeyed',
 'to',
 'the',
 'bedroom',
 '.']

In [61]:
test_data[0][1]

['Is', 'Sandra', 'in', 'the', 'hallway', '?']

In [62]:
test_data[0][2]

'no'

In [57]:
predicted[0]

array([1.53399460e-10, 1.43680595e-10, 1.02657063e-10, 8.23168964e-11,
       1.37666989e-10, 7.51098073e-11, 1.30027364e-10, 1.49136550e-10,
       8.54901358e-11, 8.11540696e-11, 1.56708979e-10, 2.00234759e-10,
       1.39040515e-10, 9.99991298e-01, 2.07058357e-10, 1.45635143e-10,
       9.12803583e-11, 6.98734334e-11, 8.79176801e-11, 9.65098834e-11,
       9.59941987e-11, 1.04757696e-10, 8.31104421e-11, 7.97603789e-11,
       8.74861507e-06, 9.90020912e-11, 9.82728829e-11, 9.70801772e-11,
       8.83144877e-11, 2.08386405e-10, 1.32151859e-10, 6.89543561e-11,
       1.05637533e-10, 7.91458774e-11, 1.30083430e-10, 1.03530511e-10,
       1.06115734e-10, 1.44493792e-10], dtype=float32)

In [60]:
val_max=np.argmax(predicted[0])

In [65]:
for key ,val in tknizer.word_index.items():
  if val==val_max:
    print(key)

no


In [66]:
predicted[0][val_max]

0.9999913

In [ ]:
my_story='John left the kitchen . Sandra dropped the football in the garden .'  #Testing from a vocabset fixed on the training set 
my_story.split()
my_question='Is the football in the garden ?'
my_question.split()

my_data=[(my_story.split(),my_question.split(),'yes')]

my_story,my_ques,my_ans=vectorize_stories((my_data))

pred_mys=model.predict([my_story,my_ques])
val_max=np.argmax(pred_mys[0])

for k,v in tknizer.word_index.items():
    if v==val_max:
        print(k)
       
print(z) 